<a href="https://colab.research.google.com/github/aisaza609/Datos_a_la_U_grupo_2/blob/main/Analisis_Decriptivo_de_Traslados_en_los_Hospitales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import io
from io import BytesIO

%matplotlib inline

# Dataset Itagui

Usamos el dataset de itagui de: https://www.datos.gov.co/Salud-y-Protecci-n-Social/Morbilidad-por-EAPB-2021/echz-i3bm/about_data. El hospital es el ESE Hospital San Rafael Itagüí Antioquia.

In [ ]:
# Read dataset

# URL base de la API
base_url3='https://www.datos.gov.co/resource/echz-i3bm.csv'

# Parámetros para paginación
limit = 1000  # Máximo número de filas por consulta
offset = 0  # Iniciar desde la primera fila
all_data = []  # Lista para almacenar los bloques de datos

# Bucle para descargar el dataset completo
while True:
    # Configurar los parámetros para la consulta
    params = {
        '$limit': limit,
        '$offset': offset
    }

    # Realizar la solicitud a la API con paginación
    response = requests.get(base_url3, params=params)

    # Convertir la respuesta a texto y luego a DataFrame
    data = pd.read_csv(io.StringIO(response.text))

    # Si el bloque de datos está vacío, hemos llegado al final
    if data.empty:
        break

    # Agregar el bloque de datos a la lista
    all_data.append(data)

    # Aumentar el offset para el siguiente bloque
    offset += limit

# Combinar todos los bloques en un solo DataFrame
tg_itagui = pd.concat(all_data, ignore_index=True)

# Muestra la base de datos
tg_itagui

In [ ]:
tg_itagui.columns.tolist() #mostramos los nombres de las columnas

In [ ]:
print(" \nDataset shape (rows, cols): \n\n",
      tg_itagui.shape)
print(" \nCount total NaN at each column in the dataset : \n\n",
      tg_itagui.isnull().sum())

In [ ]:
porcentaje_nulos = tg_itagui['COD_DX'].isnull().mean() * 100 ##Ver el porcentaje de nulos en la columna
print(f'Porcentaje de nulos: {porcentaje_nulos:.2f}%')
print(tg_itagui.COD_DX.isnull().sum())

In [ ]:
tg_itagui.ENTIDAD.value_counts() #mostramos la informacion de las columnas

In [ ]:
import pandas as pd

# Lista de palabras clave para identificar aseguradoras en el régimen del hospital
aseguradoras_keywords = [
    'POSITIVA', 'COLMENA', 'EQUIDAD', 'BOLIVAR', 'SURA', 'SOLIDARIA',
    'MUNDIAL', 'QBE', 'GENERALI', 'LIBERTY', 'COLPATRIA', 'SOAT'
]

# Lista de nombres específicos de empresas que están en el régimen
empresas_especificas = [
    'NUEVA EPS', 'ALIANZA MEDELLIN ANTIOQUIA SAS'
]

# Crear la columna 'RegimenHospital' y asignar 1 a las entidades que contienen las palabras clave o están en la lista específica
tg_itagui['RegimenHospital'] = tg_itagui['ENTIDAD'].apply(
    lambda x: 1 if any(keyword in x for keyword in aseguradoras_keywords) or x in empresas_especificas else 0
)

# Verificar las entidades que ahora están dentro del régimen
entidades_regimen_actualizado = tg_itagui[tg_itagui['RegimenHospital'] == 1]['ENTIDAD'].unique()
print("Entidades en el régimen:", entidades_regimen_actualizado)




In [ ]:
# Obtener la lista única de entidades que actualmente tienen el valor 0 en 'RegimenHospital'
entidades_no_regimen = tg_itagui[tg_itagui['RegimenHospital'] == 0]['ENTIDAD'].unique()

# Mostrar la lista de entidades que no están en el régimen
print("Entidades que no están en el régimen:", entidades_no_regimen)


In [ ]:
# Calcular el porcentaje de entidades que no están en el régimen (RegimenHospital = 0)
porcentaje_no_regimen = (tg_itagui['RegimenHospital'].value_counts(normalize=True)[0]) * 100

# Mostrar el porcentaje
print("Porcentaje de entidades no incluidas en el régimen:", porcentaje_no_regimen)


In [ ]:
import matplotlib.pyplot as plt

# Calcular el porcentaje de entidades en régimen y fuera de régimen
porcentaje_en_regimen = (tg_itagui['RegimenHospital'].value_counts(normalize=True)[1]) * 100
porcentaje_fuera_regimen = (tg_itagui['RegimenHospital'].value_counts(normalize=True)[0]) * 100

# Crear el gráfico de comparación
fig, ax = plt.subplots(figsize=(8, 6))

# Gráfico de barras con porcentajes
ax.bar(['En Régimen', 'Fuera de Régimen'], [porcentaje_en_regimen, porcentaje_fuera_regimen], color=['blue', 'orange'])
ax.set_title('Porcentaje de Entidades en Régimen vs. Fuera de Régimen')
ax.set_ylabel('Porcentaje')

# Añadir etiquetas de porcentaje encima de cada barra
for i, v in enumerate([porcentaje_en_regimen, porcentaje_fuera_regimen]):
    ax.text(i, v + 1, f"{v:.2f}%", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()



# DATASET DE PASTO

In [ ]:
# Read dataset

# URL base de la API
base_url4='https://www.datos.gov.co/resource/emmz-uf6m.csv'

# Parámetros para paginación
limit = 1000  # Máximo número de filas por consulta
offset = 0  # Iniciar desde la primera fila
all_data = []  # Lista para almacenar los bloques de datos

# Bucle para descargar el dataset completo
while True:
    # Configurar los parámetros para la consulta
    params = {
        '$limit': limit,
        '$offset': offset
    }

    # Realizar la solicitud a la API con paginación
    response = requests.get(base_url4, params=params)

    # Convertir la respuesta a texto y luego a DataFrame
    data = pd.read_csv(io.StringIO(response.text))

    # Si el bloque de datos está vacío, hemos llegado al final
    if data.empty:
        break

    # Agregar el bloque de datos a la lista
    all_data.append(data)

    # Aumentar el offset para el siguiente bloque
    offset += limit

# Combinar todos los bloques en un solo DataFrame
tg_pasto = pd.concat(all_data, ignore_index=True)

# Muestra la base de datos
tg_pasto



In [ ]:
df_pasto.columns.tolist() #mostramos los nombres de las columnas

In [ ]:
df_pasto.Objeto_Remision.value_counts() #mostramos la informacion de las columnas

In [ ]:
# Agregar una nueva columna 'NivelHospital' con el valor 1 en todo el dataset
df_pasto['NivelHospital'] = 1

# Mostrar las primeras filas para verificar la nueva columna
df_pasto.head()


In [ ]:

# Definir el mapeo de especialidades a niveles de atención
nivel_atencion_map = {
    "nivel 1": ["medicina general", "nutricion", "optometria", "psicologia", "terapia fisica", "rehabilitacion"],
    "nivel 2": ["medicina interna", "cirugia general", "ortopedia", "ginecologia", "pediatria", "urologia", "psiquiatria", "alergologia"],
    "nivel 3": ["cardiologia", "cirugia plastica", "cirugia vascular", "dermatologia", "endocrinologia", "fisiatria",
                "gastroenterologia", "genetica", "hematologia", "nefrologia", "neumologia", "neurocirugia",
                "neurologia", "neuropediatria", "oftalmologia", "oncologia", "otorrinolaringologia", "reumatologia"]
}

# Función para asignar el nivel de atención según el objeto de remisión
def asignar_nivel_atencion(objeto_remision):
    objeto_remision_lower = str(objeto_remision).lower()  # Convertir a minúsculas para comparación
    for nivel, especialidades in nivel_atencion_map.items():
        if any(especialidad in objeto_remision_lower for especialidad in especialidades):
            return int(nivel.split()[-1])  # Devolver solo el número de nivel
    return None  # Si no se encuentra, devolver None

# Aplicar la función para crear la columna 'NiveldeAtencionNecesario'
df_pasto['NiveldeAtencionNecesario'] = df_pasto['Objeto_Remision'].apply(asignar_nivel_atencion)

# Guardar el dataset actualizado
df_pasto.to_csv('Remisiones_de_pacientes_pasto_con_nivel.csv', index=False)

# Mostrar el conteo de niveles asignados para verificar
print(df_pasto['NiveldeAtencionNecesario'].value_counts())



In [ ]:
df_pasto.NiveldeAtencionNecesario.value_counts() #mostramos la informacion de las columnas

In [ ]:
import matplotlib.pyplot as plt

# Definir el nivel del hospital
nivel_hospital = 1

# Crear la columna 'Traslado' con la lógica actualizada
def definir_traslado(nivel_necesario):
    if nivel_necesario == nivel_hospital:
        return 0  # No se necesita traslado
    elif nivel_necesario == 2:
        return 1  # Traslado a un hospital de nivel 2
    elif nivel_necesario == 3:
        return 2  # Traslado a un hospital de nivel 3
    return None  # Para casos donde el nivel no esté definido

# Aplicar la función para crear la columna 'Traslado'
df_pasto['Traslado'] = df_pasto['NiveldeAtencionNecesario'].apply(definir_traslado)

# Calcular los porcentajes de cada tipo de traslado
porcentaje_no_traslado = (df_pasto['Traslado'].value_counts(normalize=True)[0]) * 100
porcentaje_traslado_nivel_2 = (df_pasto['Traslado'].value_counts(normalize=True).get(1, 0)) * 100
porcentaje_traslado_nivel_3 = (df_pasto['Traslado'].value_counts(normalize=True).get(2, 0)) * 100

# Crear un DataFrame de comparación para visualización
comparacion_traslado = pd.DataFrame({
    'Categoria': ['Sin Traslado', 'Traslado a Nivel 2', 'Traslado a Nivel 3'],
    'Porcentaje': [porcentaje_no_traslado, porcentaje_traslado_nivel_2, porcentaje_traslado_nivel_3]
})

# Gráfico de comparación de porcentajes de traslado
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(comparacion_traslado['Categoria'], comparacion_traslado['Porcentaje'], color=['green', 'orange', 'red'])
ax.set_title('Porcentaje de Traslados Necesarios')
ax.set_ylabel('Porcentaje')

# Añadir etiquetas de porcentaje encima de cada barra
for i, v in enumerate(comparacion_traslado['Porcentaje']):
    ax.text(i, v + 1, f"{v:.2f}%", ha='center', fontweight='bold')

plt.show()

# Verificar la nueva columna y el porcentaje de traslados
print("Porcentaje sin traslado:", porcentaje_no_traslado)
print("Porcentaje traslado a nivel 2:", porcentaje_traslado_nivel_2)
print("Porcentaje traslado a nivel 3:", porcentaje_traslado_nivel_3)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponiendo que tienes una columna 'Edad' y ya tienes 'Traslado' en el dataset
# Crear los rangos de edad
df_pasto['RangoEdad'] = pd.cut(df_pasto['Edad'], bins=[0, 17, 35, 50, 65, 100],
                               labels=['Menores de 18', '18-35', '36-50', '51-65', 'Mayores de 65'])

# Calcular el porcentaje de cada tipo de traslado por rango de edad
traslados_por_edad = df_pasto.groupby(['RangoEdad', 'Traslado']).size().unstack(fill_value=0)
traslados_por_edad = traslados_por_edad.div(traslados_por_edad.sum(axis=1), axis=0) * 100

# Crear el gráfico
traslados_por_edad.plot(kind='bar', stacked=True, color=['green', 'orange', 'red'], figsize=(10, 6))
plt.title('Porcentaje de Traslados por Rango de Edad y Nivel de Atención Requerido')
plt.xlabel('Rango de Edad')
plt.ylabel('Porcentaje')
plt.legend(['Sin Traslado', 'Traslado a Nivel 2', 'Traslado a Nivel 3'], title='Tipo de Traslado')
plt.show()


In [ ]:
# Calcular los porcentajes totales de cada tipo de traslado en el dataset completo para verificar la distribución general
traslado_counts = df_pasto['Traslado'].value_counts(normalize=True) * 100

# Verificar la cantidad de cada tipo de traslado (sin traslado, traslado a nivel 2, traslado a nivel 3) en porcentaje
porcentaje_sin_traslado = traslado_counts.get(0, 0)  # Porcentaje de sin traslado
porcentaje_traslado_nivel_2 = traslado_counts.get(1, 0)  # Porcentaje de traslado a nivel 2
porcentaje_traslado_nivel_3 = traslado_counts.get(2, 0)  # Porcentaje de traslado a nivel 3

porcentaje_sin_traslado, porcentaje_traslado_nivel_2, porcentaje_traslado_nivel_3


In [ ]:
# Crear una nueva columna 'RangoEdad' con los grupos de edad especificados
# 1: Menor de 18 años, 2: Entre 19 y 45 años, 3: 45 años en adelante

df_pasto['RangoEdad'] = pd.cut(df_pasto['Edad'], bins=[0, 18, 45, 100], labels=[1, 2, 3], right=False)

# Verificar la nueva columna para asegurarnos de que se creó correctamente
df_pasto[['Edad', 'RangoEdad']].head(10)


In [ ]:
df_pasto.RangoEdad.value_counts() #mostramos la informacion de las columnas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Calcular el porcentaje de cada tipo de traslado dentro de cada rango de edad
traslados_por_rango = df_pasto.groupby(['RangoEdad', 'Traslado']).size().unstack(fill_value=0)
traslados_por_rango = traslados_por_rango.div(traslados_por_rango.sum(axis=1), axis=0) * 100

# Configuración de las etiquetas y los colores
categorias = ['Sin Traslado', 'Traslado a Nivel 2', 'Traslado a Nivel 3']
bar_width = 0.25
index = np.arange(len(traslados_por_rango))

fig, ax = plt.subplots(figsize=(10, 6))

# Crear barras para cada tipo de traslado en cada rango de edad
for i, categoria in enumerate(categorias):
    plt.bar(index + i * bar_width, traslados_por_rango[i], bar_width, label=categoria)

# Configurar etiquetas y diseño
ax.set_xlabel('Rango de Edad')
ax.set_ylabel('Porcentaje de Traslados')
ax.set_title('Distribución de Traslados por Rango de Edad')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(['Menores de 18', '18-45', '45 en adelante'])
ax.legend(title="Tipo de Traslado")

# Ajustar el diseño de fondo
fig.patch.set_facecolor('#DDE3FA')
ax.set_facecolor('#F3F3FF')

# Mostrar la gráfica
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Calcular el porcentaje de cada tipo de traslado dentro de cada rango de edad
traslados_por_rango = df_pasto.groupby(['RangoEdad', 'Traslado']).size().unstack(fill_value=0)
traslados_por_rango = traslados_por_rango.div(traslados_por_rango.sum(axis=1), axis=0) * 100

# Configuración de las etiquetas y los colores
categorias = ['Sin Traslado', 'Traslado a Nivel 2', 'Traslado a Nivel 3']
bar_width = 0.25
index = np.arange(len(traslados_por_rango))

fig, ax = plt.subplots(figsize=(10, 6))

# Crear barras para cada tipo de traslado en cada rango de edad
for i, categoria in enumerate(categorias):
    barras = plt.bar(index + i * bar_width, traslados_por_rango[i], bar_width, label=categoria)

    # Agregar etiquetas de porcentaje encima de cada barra
    for bar in barras:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval + 1, f"{yval:.2f}%", ha='center', va='bottom', fontweight='bold')

# Configurar etiquetas y diseño
ax.set_xlabel('Rango de Edad')
ax.set_ylabel('Porcentaje de Traslados')
ax.set_title('Distribución de Traslados por Rango de Edad')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(['Menores de 18', '18-45', '45 en adelante'])
ax.legend(title="Tipo de Traslado")

# Ajustar el diseño de fondo
fig.patch.set_facecolor('#DDE3FA')
ax.set_facecolor('#F3F3FF')

# Mostrar la gráfica
plt.tight_layout()
plt.show()


In [ ]:
# Ordenar la tabla por el porcentaje de traslados a Nivel 2 y Nivel 3 y filtrar los de mayor frecuencia (por ejemplo, los primeros 5 de cada nivel)

# Ordenar y filtrar por porcentaje de traslados a Nivel 2
top_nivel_2 = tabla_traslados_por_nivel[['Objeto_Remision', 'Porcentaje Nivel 2']].sort_values(by='Porcentaje Nivel 2', ascending=False).head(5)

# Ordenar y filtrar por porcentaje de traslados a Nivel 3
top_nivel_3 = tabla_traslados_por_nivel[['Objeto_Remision', 'Porcentaje Nivel 3']].sort_values(by='Porcentaje Nivel 3', ascending=False).head(5)

# Mostrar las tablas ordenadas
top_nivel_2, top_nivel_3



In [ ]:
import pandas as pd

# Calcular el porcentaje de cada tipo de traslado por especialidad médica
traslados_por_especialidad = df_pasto.groupby(['Objeto_Remision', 'Traslado']).size().unstack(fill_value=0)
traslados_por_especialidad = traslados_por_especialidad.div(traslados_por_especialidad.sum(axis=1), axis=0) * 100

# Mostrar la tabla resultante
print("Porcentaje de Traslados por Especialidad Médica:")
print(traslados_por_especialidad)


In [ ]:
import pandas as pd

# Calcular el total de pacientes por especialidad
traslados_por_especialidad_total = df_pasto.groupby('Objeto_Remision').size()
traslados_por_especialidad = df_pasto.groupby(['Objeto_Remision', 'Traslado']).size().unstack(fill_value=0)

# Calcular los porcentajes de traslados por especialidad
traslados_por_especialidad['Total Pacientes'] = traslados_por_especialidad_total
traslados_por_especialidad['Porcentaje Sin Traslado'] = (traslados_por_especialidad[0] / traslados_por_especialidad['Total Pacientes']) * 100
traslados_por_especialidad['Porcentaje Traslado Nivel 2'] = (traslados_por_especialidad[1] / traslados_por_especialidad['Total Pacientes']) * 100
traslados_por_especialidad['Porcentaje Traslado Nivel 3'] = (traslados_por_especialidad[2] / traslados_por_especialidad['Total Pacientes']) * 100

# Seleccionar y organizar las columnas relevantes para la tabla final
tabla_traslados = traslados_por_especialidad[['Total Pacientes', 'Porcentaje Sin Traslado', 'Porcentaje Traslado Nivel 2', 'Porcentaje Traslado Nivel 3']]
tabla_traslados.reset_index(inplace=True)

# Mostrar la tabla final
print("Tabla de Porcentajes de Traslados por Especialidad:")
print(tabla_traslados)

